In [7]:
import main
main.create_csv_from_las()

AttributeError: module 'main' has no attribute 'create_csv_from_las'

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv(main.csv_out_file, delimiter=';')
df.describe()